<a href="https://colab.research.google.com/github/dcolinmorgan/grph/blob/main/cu_cat_final_bench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install --extra-index-url=https://pypi.nvidia.com cuml-cu11==22.10.0 cudf-cu11==22.10.0 cugraph-cu11==22.10.0 pylibraft_cu11==22.10.0 raft_dask_cu11==22.10.0 dask_cudf_cu11==22.10.0 pylibcugraph_cu11==22.10.0 pylibraft_cu11==22.10.0
!pip install -U git+https://github.com/graphistry/pygraphistry.git@cudf-cat-final
!pip install -U git+https://github.com/graphistry/cu-cat.git@cu_cat_regpt

In [4]:
# !pip install dirty_cat

In [6]:
import cuml,cudf
cuml.__version__

'22.10.00a+119.gb30ef85c1'

In [7]:
!echo "0.01.0" > VERSION.txt
# !cp VERSION.txt /root/.local/lib/python3.9/site-packages/cu_cat/
!cp VERSION.txt /usr/local/lib/python3.9/dist-packages/cu_cat/

In [8]:
# from importlib import reload
# reload(cu_cat)
import cu_cat
print(cu_cat.__file__)

/usr/local/lib/python3.9/dist-packages/cu_cat/__init__.py


In [9]:
import os
from collections import Counter
import cProfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pstats import Stats
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 200)


In [11]:
import graphistry
graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username='dcolinmorgan', password='***') ## key id, secret key
graphistry.__version__


'0.28.7+393.g2e9820c'

In [ ]:
# import logging
# logging.basicConfig(level=logging.DEBUG)

### redteam

In [9]:
df = pd.read_csv('https://gist.githubusercontent.com/silkspace/c7b50d0c03dc59f63c48d68d696958ff/raw/31d918267f86f8252d42d2e9597ba6fc03fcdac2/redteam_50k.csv', index_col=0)
red_team = pd.read_csv('https://gist.githubusercontent.com/silkspace/5cf5a94b9ac4b4ffe38904f20d93edb1/raw/888dabd86f88ea747cf9ff5f6c44725e21536465/redteam_labels.csv', index_col=0)
df['feats'] = df.src_computer + ' ' + df.dst_computer + ' ' + df.auth_type + ' ' + df.logontype
df['feats2'] = df.src_computer + ' ' + df.dst_computer
ndf = df.drop_duplicates(subset=['feats'])
tdf = pd.concat([red_team.reset_index(), ndf.reset_index()])
tdf['node'] = range(len(tdf))

749.0

In [ ]:
%%time
# process = True  # set to false after it's run for ease of speed
pr = cProfile.Profile()
pr.enable()

g = graphistry.nodes(cudf.from_pandas(tdf), 'node')

g5b = g.umap(X=['feats'], 
            min_words=1000000, # force high so that we don't use Sentence Transformers
            cardinality_threshold=4, # set low so we force topic model
            n_topics=32, # number of topics
            use_scaler=None,
            feature_engine='cu_cat',
            memoize=False,
            #  remove_node_column=False,
            # engine='cuml',
            use_scaler_target=None
           )

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('redteam_umap_cudf_cu_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

In [11]:
%%time
# process = True  # set to false after it's run for ease of speed
pr = cProfile.Profile()
pr.enable()

g = graphistry.nodes(tdf, 'node')

g5a = g.umap(X=['feats'], 
            min_words=1000000, # force high so that we don't use Sentence Transformers
            cardinality_threshold=4, # set low so we force topic model
            n_topics=32, # number of topics
            use_scaler=None,
            feature_engine='cu_cat',
            memoize=False,
             remove_node_column=False,
            # engine='cuml',
            use_scaler_target=None
           )

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('redteam_umap_cu_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

Using GPU: cu_cat


         1624287 function calls (1608282 primitive calls) in 41.368 seconds

   Ordered by: internal time
   List reduced from 2359 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        8   20.256    2.532   24.775    3.097 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:1126(_multiplicative_update_h_smallfast)
        3   12.664    4.221   12.664    4.221 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
     4236    4.104    0.001    4.156    0.001 /usr/local/lib/python3.9/dist-packages/rmm/rmm.py:216(rmm_cupy_allocator)
        7    1.132    0.162    1.388    0.198 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:181(_get_H)
      213    0.458    0.002    0.711    0.003 /usr/local/lib/python3.9/dist-packages/cupyx/scipy/sparse/csr.py:148(__mul__)
      478    0.411    0.001    0.448    0.001 {method 'update' of 'dict' objects}
        5    0.309    0.062    0.667    0.133 /usr

In [ ]:
!pip install dirty_cat

In [15]:
%%time
# process = True  # set to false after it's run for ease of speed
pr = cProfile.Profile()
pr.enable()

g = graphistry.nodes((tdf), 'node')

g5c = g.umap(X=['feats'], 
            min_words=1000000, # force high so that we don't use Sentence Transformers
            cardinality_threshold=4, # set low so we force topic model
            n_topics=32, # number of topics
            use_scaler=None,
            feature_engine='dirty_cat',
            memoize=False,
            #  remove_node_column=False,
            # engine='cuml',
            use_scaler_target=None
           )

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('redteam_umap_dirty_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

Using GPU: dirty_cat


         28960368 function calls (28800717 primitive calls) in 110.439 seconds

   Ordered by: internal time
   List reduced from 2562 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
2916339/2916096   22.138    0.000   39.057    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
     1054   17.698    0.017   48.474    0.046 /usr/local/lib/python3.9/dist-packages/dirty_cat/_gap_encoder.py:1003(_multiplicative_update_h)
        3   16.647    5.549   16.647    5.549 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
    39524    8.337    0.000   10.021    0.000 /usr/local/lib/python3.9/dist-packages/pandas/core/missing.py:61(mask_missing)
      775    4.697    0.006   14.330    0.018 /usr/local/lib/python3.9/dist-packages/dirty_cat/_gap_encoder.py:971(_multiplicative_update_w)
   272946    3.786    0.000    8.226    0.000 /usr/local/lib/python3.9/dist-packages/scipy/sparse/_sputils.py:

### CTU-13 malware dataset

In [12]:
edf = pd.read_csv('https://gist.githubusercontent.com/silkspace/33bde3e69ae24fee1298a66d1e00b467/raw/dc66bd6f1687270be7098f94b3929d6a055b4438/malware_bots.csv', index_col=0)
T = edf.Label.apply(lambda x: True if 'Botnet' in x else False)
bot = edf[T]
nbot = edf[~T]
print(f'Botnet abundance: {100*len(bot)/len(edf):0.2f}%')# so botnet traffic makes up a tiny fraction of total

# let's balance the dataset in a 10-1 ratio, for speed and demonstrative purposes
negs = nbot.sample(10*len(bot))
edf = pd.concat([bot, negs])  # top part of arrays are bot traffic, then all non-bot traffic
edf = edf.drop_duplicates()

# some useful indicators for later that predict Botnet as Bool and Int
Y = edf.Label.apply(lambda x: 1 if 'Botnet' in x else 0)  # np.array(T)

# Later we will use and exploit any meaning shared between the labels in a latent distribution

# add it to the dataframe
edf['bot'] = Y

# name some columns for edges and features
src = 'SrcAddr'
dst = 'DstAddr'
good_cols_with_edges = ['Dur', 'Proto', 'Sport',
       'Dport', 'State', 'TotBytes', 'SrcBytes', src, dst]

good_cols_without_edges = ['Dur', 'Proto', 'Sport',
       'Dport', 'State', 'TotPkts', 'TotBytes', 'SrcBytes']

## some encoding parameters
n_topics = 20
n_topics_target = 7



Botnet abundance: 0.69%


In [14]:
# lets umap the data
g = graphistry.edges(cudf.from_pandas(edf), src, dst) ## something with umap cudf still off a bit, so ran featurize
pr = cProfile.Profile()
pr.enable()
g1a = g.featurize(kind='edges', 
            X=good_cols_with_edges, 
            y = ['bot'], 
            # use_scaler='quantile',
            # use_scaler_target=None,
            cardinality_threshold=20,
            cardinality_threshold_target=2, 
            n_topics=n_topics,
            feature_engine='cu_cat',
            # engine='cuml',
            memoize=False,
            n_topics_target=n_topics_target,
            n_bins=n_topics_target,
            # metric='euclidean', 
            # n_neighbors=12)
)
pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('ctu13_umap_cu_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)


Featurizing nodes with feature_engine=cu_cat
Using GPU: cu_cat
         9177809 function calls (9076803 primitive calls) in 53.577 seconds

   Ordered by: internal time
   List reduced from 2274 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       81   23.140    0.286   23.140    0.286 {built-in method cupy_backends.cuda.libs.nvrtc.compileProgram}
       40    6.209    0.155   13.838    0.346 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:1126(_multiplicative_update_h_smallfast)
       15    4.736    0.316    4.736    0.316 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
      677    3.505    0.005    3.510    0.005 {built-in method cupy._core._routines_linalg.matmul}
    17837    1.677    0.000    1.904    0.000 /usr/local/lib/python3.9/dist-packages/rmm/rmm.py:216(rmm_cupy_allocator)
       35    1.643    0.047    1.974    0.056 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.

In [15]:
# lets umap the data
g = graphistry.edges(edf, src, dst)

pr = cProfile.Profile()
pr.enable()
g6a = g.featurize(kind='edges', 
            X=good_cols_with_edges, 
            y = ['bot'], 
            use_scaler='quantile',
            use_scaler_target=None,
            cardinality_threshold=20,
            cardinality_threshold_target=2, 
            n_topics=n_topics,
            feature_engine='cu_cat',
            # engine='cuml',
            memoize=False,
            n_topics_target=n_topics_target,
            n_bins=n_topics_target,
            # metric='euclidean', 
            # n_neighbors=12)
          )
pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('ctu13_umap_cu_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)


Featurizing nodes with feature_engine=cu_cat
Using GPU: cu_cat
         14364844 function calls (14198592 primitive calls) in 50.806 seconds

   Ordered by: internal time
   List reduced from 2484 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    71288    6.596    0.000    6.669    0.000 {built-in method numpy.array}
       40    6.122    0.153    7.931    0.198 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:1126(_multiplicative_update_h_smallfast)
       15    4.827    0.322    4.828    0.322 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
    13168    2.666    0.000    2.666    0.000 {method 'take' of 'numpy.ndarray' objects}
      208    2.120    0.010    2.120    0.010 {method 'argsort' of 'numpy.ndarray' objects}
        3    1.883    0.628    1.883    0.628 /usr/local/lib/python3.9/dist-packages/numpy/ma/core.py:3211(__getitem__)
       35    1.702    0.049    2.054    0.059 /usr/loc

In [ ]:
!pip install dirty_cat

In [18]:
### lets umap the data
g = graphistry.edges(edf, src, dst)
pr = cProfile.Profile()
pr.enable()
g6c = g.featurize(kind='edges', 
            X=good_cols_with_edges, 
            y = ['bot'], 
            use_scaler='quantile',
            use_scaler_target=None,
            cardinality_threshold=20,
            cardinality_threshold_target=2, 
            n_topics=n_topics,
            feature_engine='dirty_cat',
            # engine='cuml',
            memoize=False,
            n_topics_target=n_topics_target,
            n_bins=n_topics_target,
            # metric='euclidean', 
            # n_neighbors=12)
          )
pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('ctu13_umap_dirtycat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)


Featurizing nodes with feature_engine=dirty_cat
Using GPU: dirty_cat
         16859133 function calls (16637906 primitive calls) in 119.510 seconds

   Ordered by: internal time
   List reduced from 2281 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   348584   19.555    0.000   20.112    0.000 {built-in method numpy.array}
1719805/1663063   13.469    0.000   37.938    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
     1389   10.500    0.008   34.249    0.025 /usr/local/lib/python3.9/dist-packages/dirty_cat/_gap_encoder.py:1003(_multiplicative_update_h)
        9    8.427    0.936    8.427    0.936 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
        3    7.004    2.335    7.004    2.335 /usr/local/lib/python3.9/dist-packages/numpy/ma/core.py:3211(__getitem__)
    13122    6.933    0.001    6.933    0.001 {method 'take' of 'numpy.ndarray' objects}
      143    5.675    

### 20newsgroups

In [20]:
from sklearn.datasets import fetch_20newsgroups
# newsgroups = fetch_20newsgroups()#categories=categories)
n_samples = 500
# n_features = 1000
# n_components = 10
# n_top_words = 20
# batch_size = 128
# init = "nndsvda"

news, _ = fetch_20newsgroups(
    shuffle=True,
    random_state=1,
    remove=("headers", "footers", "quotes"),
    return_X_y=True,
)

news = news[:n_samples]
news=pd.DataFrame(news)

In [21]:
import logging
# logging.basicConfig(level=logging)
logging.basicConfig(level=None)

In [22]:
g = graphistry.nodes(cudf.from_pandas(news))
pr = cProfile.Profile()
pr.enable()


g8 = g.featurize(feature_engine='cu_cat',memoize=False)
g8b = g8.umap()#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('20news_umap_cudf-cu_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

Featurizing nodes with feature_engine=cu_cat
Using GPU: cu_cat


Using GPU: cu_cat


         17140650 function calls (17107731 primitive calls) in 90.004 seconds

   Ordered by: internal time
   List reduced from 2511 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      416   12.858    0.031   46.479    0.112 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:1042(_multiplicative_update_h)
       12   11.673    0.973   14.952    1.246 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:1126(_multiplicative_update_h_smallfast)
        6   10.637    1.773   10.637    1.773 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
   168234    7.370    0.000    8.834    0.000 /usr/local/lib/python3.9/dist-packages/rmm/rmm.py:216(rmm_cupy_allocator)
    26748    5.594    0.000    8.341    0.000 /usr/local/lib/python3.9/dist-packages/cupyx/scipy/sparse/_index.py:44(_get_csr_submatrix_major_axis)
       14    3.202    0.229    7.352    0.525 /usr/local/lib/python3.9/dist-packages/cu

In [23]:
g = graphistry.nodes(news)
pr = cProfile.Profile()
pr.enable()

g7 = g.featurize(feature_engine='cu_cat',memoize=False)
g7a = g7.umap()#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('20news_umap_cu_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

Featurizing nodes with feature_engine=cu_cat
Using GPU: cu_cat


Using GPU: cu_cat


         17164380 function calls (17131488 primitive calls) in 84.611 seconds

   Ordered by: internal time
   List reduced from 2433 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       12   11.993    0.999   15.394    1.283 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:1126(_multiplicative_update_h_smallfast)
      416   11.639    0.028   39.970    0.096 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:1042(_multiplicative_update_h)
        6    9.901    1.650    9.901    1.650 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
   168234    7.289    0.000    8.759    0.000 /usr/local/lib/python3.9/dist-packages/rmm/rmm.py:216(rmm_cupy_allocator)
    26748    4.437    0.000    6.471    0.000 /usr/local/lib/python3.9/dist-packages/cupyx/scipy/sparse/_index.py:44(_get_csr_submatrix_major_axis)
       14    3.863    0.276   10.062    0.719 /usr/local/lib/python3.9/dist-packages/cu

In [ ]:
g = graphistry.nodes(news)
pr = cProfile.Profile()
pr.enable()

g9 = g.featurize(feature_engine='dirty_cat',memoize=False)
g9c = g9.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('20news_umap_dirtycat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

Featurizing nodes with feature_engine=dirty_cat
Using GPU: dirty_cat


### winlogs

In [8]:
!wget -nc https://www.dropbox.com/s/31dx1g6g59exoc3/part.88.parquet
winlogs=pd.read_parquet('part.88.parquet')
n=100000
winlogs=winlogs.sample(n,replace=False)#.iloc[:,:8]
winlogs=winlogs.dropna(how='any',axis=1)

g = graphistry.nodes(winlogs)
pr = cProfile.Profile()
pr.enable()

g10 = g.featurize(feature_engine='cu_cat',memoize=False)
# g10a = g10.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('winlogs_featurize+umap_cu_cat_'+str(n)+'.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

g = graphistry.nodes(cudf.from_pandas(winlogs))
pr = cProfile.Profile()
pr.enable()

g11 = g.featurize(feature_engine='cu_cat',memoize=False)
# g11b = g11.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('winlogs_featurize+umap_cudf-cu_cat_'+str(n)+'.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)


g = graphistry.nodes(winlogs)
pr = cProfile.Profile()
pr.enable()


g12 = g.featurize(feature_engine='dirty_cat',memoize=False)
# g12c = g12.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('winlogs_featurize+umap_dirtycat_'+str(n)+'.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

File ‘part.88.parquet’ already there; not retrieving.

Featurizing nodes with feature_engine=cu_cat
Using GPU: cu_cat


         4766439 function calls (4726326 primitive calls) in 74.039 seconds

   Ordered by: internal time
   List reduced from 2050 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       24   37.937    1.581   42.562    1.773 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:1126(_multiplicative_update_h_smallfast)
        9   18.896    2.100   18.897    2.100 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
       21    5.809    0.277    6.994    0.333 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:181(_get_H)
    10329    3.731    0.000    3.864    0.000 /usr/local/lib/python3.9/dist-packages/rmm/rmm.py:216(rmm_cupy_allocator)
       15    0.976    0.065    1.116    0.074 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:994(_multiplicative_update_w_smallfast)
   456139    0.624    0.000    0.646    0.000 {built-in method cupy._core.core.array}
     1065    0.608    

In [9]:
print(winlogs.shape)


(100000, 5)


### ask HN

In [28]:
# # get the data top 3000 posts on Hacker News
askHN = pd.read_csv('https://storage.googleapis.com/cohere-assets/blog/text-clustering/data/askhn3k_df.csv', index_col=0)

# askHN = askHN.sample(1000,replace=False) 
# df = df[['text','title']]
# df['title'] = df['title'].sample(frac=1).values

In [29]:
g = graphistry.nodes(cudf.from_pandas(askHN))
pr = cProfile.Profile()
pr.enable()

g2 = g.featurize(X=['title', 'text'], # the features to encode (can add/remove 'text', etc)
                y=['score'], # for demonstrative purposes, we include a target -- though this one is not really conditioned on textual features in a straightforward way
                model_name='msmarco-distilbert-base-v2', #'paraphrase-MiniLM-L6-v2', etc, from sbert/Huggingface, the text encoding model
                min_words = 0, # when 0 forces all X=[..] as textually encoded, higher values would ascertain if a column is textual or not depending on average number of words per column
                use_ngrams=False, # set to True if you want ngram features instead (does not make great plots but useful for other situations)
                # use_scaler_target='zscale', # for regressive targets
                # use_scaler=None, # there are many more settings see `g.featurize?` and `g.umap?` for further options
                feature_engine='cu_cat',
                 memoize=False,    
               )# g1a = g1.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('aHN_umap_cudf_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

Featurizing nodes with feature_engine=cu_cat
Using GPU: cu_cat
         11133374 function calls (11102447 primitive calls) in 84.341 seconds

   Ordered by: internal time
   List reduced from 1950 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       14   41.635    2.974   47.606    3.400 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:1126(_multiplicative_update_h_smallfast)
      242    6.050    0.025   20.891    0.086 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:1042(_multiplicative_update_h)
        6    6.027    1.004    6.027    1.004 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
   101167    5.518    0.000    6.273    0.000 /usr/local/lib/python3.9/dist-packages/rmm/rmm.py:216(rmm_cupy_allocator)
      412    2.483    0.006    2.763    0.007 /usr/local/lib/python3.9/dist-packages/cupyx/scipy/sparse/csr.py:148(__mul__)
    15526    2.324    0.000    3.367    0.000 /u

In [30]:
g = graphistry.nodes(askHN)
pr = cProfile.Profile()
pr.enable()

g2 = g.featurize(X=['title', 'text'], # the features to encode (can add/remove 'text', etc)
                y=['score'], # for demonstrative purposes, we include a target -- though this one is not really conditioned on textual features in a straightforward way
                model_name='msmarco-distilbert-base-v2', #'paraphrase-MiniLM-L6-v2', etc, from sbert/Huggingface, the text encoding model
                min_words = 0, # when 0 forces all X=[..] as textually encoded, higher values would ascertain if a column is textual or not depending on average number of words per column
                use_ngrams=False, # set to True if you want ngram features instead (does not make great plots but useful for other situations)
                # use_scaler_target='zscale', # for regressive targets
                # use_scaler=None, # there are many more settings see `g.featurize?` and `g.umap?` for further options
                feature_engine='cu_cat',
                 memoize=False,    
               )# g2a = g2.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('aHN_umap_cu_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

Featurizing nodes with feature_engine=cu_cat
Using GPU: cu_cat
         11122341 function calls (11091393 primitive calls) in 85.601 seconds

   Ordered by: internal time
   List reduced from 2027 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       14   41.650    2.975   47.820    3.416 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:1126(_multiplicative_update_h_smallfast)
      242    6.654    0.027   22.132    0.091 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:1042(_multiplicative_update_h)
        6    6.509    1.085    6.509    1.085 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
   101149    5.611    0.000    6.329    0.000 /usr/local/lib/python3.9/dist-packages/rmm/rmm.py:216(rmm_cupy_allocator)
      411    2.508    0.006    2.789    0.007 /usr/local/lib/python3.9/dist-packages/cupyx/scipy/sparse/csr.py:148(__mul__)
    15526    2.274    0.000    3.284    0.000 /u

In [31]:
g = graphistry.nodes(askHN)
pr = cProfile.Profile()
pr.enable()


g2 = g.featurize(X=['title', 'text'], # the features to encode (can add/remove 'text', etc)
                y=['score'], # for demonstrative purposes, we include a target -- though this one is not really conditioned on textual features in a straightforward way
                model_name='msmarco-distilbert-base-v2', #'paraphrase-MiniLM-L6-v2', etc, from sbert/Huggingface, the text encoding model
                min_words = 0, # when 0 forces all X=[..] as textually encoded, higher values would ascertain if a column is textual or not depending on average number of words per column
                use_ngrams=False, # set to True if you want ngram features instead (does not make great plots but useful for other situations)
                # use_scaler_target='zscale', # for regressive targets
                # use_scaler=None, # there are many more settings see `g.featurize?` and `g.umap?` for further options
                feature_engine='dirty_cat',
                 memoize=False,    
               )# g3a = g3.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('aHN_umap_dirtycat_3k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

Featurizing nodes with feature_engine=dirty_cat
Using GPU: dirty_cat
         30404485 function calls (30379291 primitive calls) in 117.927 seconds

   Ordered by: internal time
   List reduced from 1561 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
3656983/3656502   36.575    0.000   44.639    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
      585   28.521    0.049   67.024    0.115 /usr/local/lib/python3.9/dist-packages/dirty_cat/_gap_encoder.py:1003(_multiplicative_update_h)
      240   11.019    0.046   28.277    0.118 /usr/local/lib/python3.9/dist-packages/dirty_cat/_gap_encoder.py:971(_multiplicative_update_w)
        6    7.101    1.183    7.101    1.184 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
       14    3.972    0.284   10.082    0.720 /usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:1258(_count_vocab)
    10080    3.421    0.000

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
!cp *.txt /content/drive/MyDrive/graphistry